In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Experiment 1: Train Logistic Regression Classifier

In [5]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



Experiment 2: Train Random Forest Classifier

In [6]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



Experiment 3: Train XGBoost

In [7]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



Handle Class Imbalance using SMOTE

In [16]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

TRACK EXPERIMENTS USING MLFLOW

In [22]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [24]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    model.set_params(**params)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [28]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [30]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection1")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/12/26 10:12:36 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection1' does not exist. Creating a new experiment.
2024/12/26 10:12:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/274110606972973547/runs/e1fd5821b6e04de69f9bb6498d5fa060
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/274110606972973547


2024/12/26 10:12:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/274110606972973547/runs/2751e395125c4c328962661e8ab6ecfb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/274110606972973547


2024/12/26 10:12:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/274110606972973547/runs/ae0b55fb7e2c4582993c6f272bfa5026
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/274110606972973547


2024/12/26 10:12:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/274110606972973547/runs/95656603d90f41e084cdeb85c4592e3b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/274110606972973547


Register the model

In [42]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 95656603d90f41e084cdeb85c4592e3b


Successfully registered model 'XGB-Smote'.
2024/12/26 10:34:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/274110606972973547/runs/95656603d90f41e084cdeb85c4592e3b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/274110606972973547


Created version '1' of model 'XGB-Smote'.


Load the model

In [48]:
model_name = 'XGB-Smote'
model_version = 1
# model_uri = f"models:/{model_name}@/challenger"
model_uri = f"models:/{model_name}/{model_version}"
try:
    # Attempt to load the model
    loaded_model = mlflow.xgboost.load_model(model_uri)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {str(e)}")

Error loading model: 'super' object has no attribute '__sklearn_tags__'


In [47]:
model_name = 'XGB-Smote'
model_version = 1
# model_uri = f"models:/{model_name}@/challenger"
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
loaded_model
# y_pred = loaded_model.predict(X_test)
# y_pred[:4]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda2024\envs\mlflow-env\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15736\8716401.py", line 6, in <module>
    loaded_model = mlflow.xgboost.load_model(model_uri)
  File "C:\Users\Lenovo\anaconda2024\envs\mlflow-env\lib\site-packages\mlflow\xgboost\__init__.py", line 340, in load_model
    return _load_model(path=local_model_path)
  File "C:\Users\Lenovo\anaconda2024\envs\mlflow-env\lib\site-packages\mlflow\xgboost\__init__.py", line 302, in _load_model
    model.load_model(xgb_model_path)
  File "C:\Users\Lenovo\anaconda2024\envs\mlflow-env\lib\site-packages\xgboost\sklearn.py", line 930, in load_model
    self._load_model_attributes(config)
  File "C:\Users\Lenovo\anaconda2024\envs\mlflow-env\lib\site-packages\xgboost\sklearn.py", line 945, in _load_model_attributes
    if is_classifi